<a href="https://colab.research.google.com/github/Miksan31/Agentic-AI/blob/main/Langchain_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain groq pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install groq

In [ ]:
from groq import Groq

# Initialize Groq client
groq_client = Groq(api_key="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")  # Replace with your actual Groq API key

In [ ]:
!pip install langchain-groq

In [ ]:
from langchain_groq import ChatGroq
# Initialize the Groq client
groq_api_key = "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"  # Replace with your actual Groq API key
llm_name = "llama3-70b-8192"  # Correct Groq model name for Llama 3 70B
groq_llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name=llm_name)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain.tools import BaseTool
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
# Example: Load dataset from user upload in Colab
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))

Saving Salary_dataset.csv to Salary_dataset (1).csv


In [ ]:
target_variable = "Salary"

In [ ]:
import pandas as pd
import json

class DataPreprocessorTool(BaseTool):
    name: str = "Data Preprocessor"
    description: str = "Cleans the data, handles missing values, removes duplicates, and performs feature engineering."

    def _run(self, df_json: str) -> str:
        try:
            # Convert JSON string to DataFrame
            df = pd.read_json(df_json, orient="split")

            # Handle missing values
            df = df.dropna()

            # Remove duplicates
            df = df.drop_duplicates()

            # Encode categorical variables (if any)
            df = pd.get_dummies(df, drop_first=True)

            # Split the dataset into training and test sets
            global X_train, X_test, y_train, y_test
            X = df.drop(columns=[target_variable])
            y = df[target_variable]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Return the preprocessed DataFrame as a JSON string
            return df.to_json(orient="split")
        except Exception as e:
            return f"Error in preprocessing: {str(e)}"

data_preprocessor_tool = DataPreprocessorTool()

In [ ]:
class ModelTrainerTool(BaseTool):
    name: str = "Model Trainer"
    description: str = "Trains a Linear Regression model using the preprocessed data."

    def _run(self, input_json: str) -> str:
        try:
            # Parse input JSON to extract X_train and y_train
            input_data = json.loads(input_json)
            X_train = pd.read_json(input_data["X_train"], orient="split")
            y_train = pd.read_json(input_data["y_train"], type="series", orient="split")

            # Train the model
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Return a success message (or serialize the model)
            return "Model trained successfully."
        except Exception as e:
            return f"Error in training: {str(e)}"

model_trainer_tool = ModelTrainerTool()

In [ ]:
class ModelEvaluatorTool(BaseTool):
    name: str = "Model Evaluator"
    description: str = "Evaluates the model's performance using RMSE and R²."

    def _run(self, input_json: str) -> str:
        try:
            # Parse input JSON to extract model, X_test, y_test
            input_data = json.loads(input_json)
            model = LinearRegression()  # Replace with actual model deserialization if needed
            X_test = pd.read_json(input_data["X_test"], orient="split")
            y_test = pd.read_json(input_data["y_test"], typ="series", orient="split")

            # Evaluate the model
            y_pred = model.predict(X_test)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            r2 = r2_score(y_test, y_pred)

            # Return evaluation results
            return json.dumps({"RMSE": rmse, "R²": r2})
        except Exception as e:
            return f"Error in evaluation: {str(e)}"

model_evaluator_tool = ModelEvaluatorTool()

In [ ]:
from langchain.agents import AgentExecutor, initialize_agent
from langchain_groq import ChatGroq
from langchain.tools import Tool

# 1. Initialize Groq LLM
groq_api_key = "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"
groq_llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192", \
    groq_api_key=groq_api_key
)

# 2. Define Tools
tools = [
    Tool.from_function(
        name="Data Preprocessor",
        description="Cleans data, handles missing values, removes duplicates, and encodes categorical variables",
        func=data_preprocessor_tool.run,
    ),
    Tool.from_function(
        name="Model Trainer",
        description="Trains a Linear Regression model",
        func=model_trainer_tool.run,
    ),
    Tool.from_function(
        name="Model Evaluator",
        description="Evaluates model performance using RMSE and R²",
        func=model_evaluator_tool.run,
    )
]

# 3. Initialize Agent
agent = initialize_agent(
    tools=tools,
    llm=groq_llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True  # Add this to prevent JSON schema errors
)

<ipython-input-12-ed3c540f96e9>:33: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
import json

In [ ]:
# Step 1: Preprocess the data
preprocessed_df_json = agent.run("Preprocess this dataset: " + df.to_json(orient="split"))

# Step 2: Train the model
train_input_json = json.dumps({
    "X_train": X_train.to_json(orient="split"),
    "y_train": y_train.to_json(orient="split")
})
trained_model_result = agent.run("Train the model: " + train_input_json)

# Step 3: Evaluate the model
evaluation_input_json = json.dumps({
    "X_test": X_test.to_json(orient="split"),
    "y_test": y_test.to_json(orient="split")
})
evaluation_results_json = agent.run("Evaluate the model: " + evaluation_input_json)

# Convert results back to Python objects
preprocessed_df = pd.read_json(preprocessed_df_json, orient="split")
evaluation_results = json.loads(evaluation_results_json)

# print("Preprocessed DataFrame:", preprocessed_df.head())
print("Evaluation Results:", evaluation_results)



> Entering new AgentExecutor chain...


<ipython-input-14-608edd689230>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  preprocessed_df_json = agent.run("Preprocess this dataset: " + df.to_json(orient="split"))


Thought: The dataset needs to be preprocessed to handle missing values, remove duplicates, and encode categorical variables.

Action: Data Preprocessor
Action Input: {"columns":["Unnamed: 0","YearsExperience","Salary"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29],"data":[[0,1.2,39344.0],[1,1.4,46206.0],[2,1.6,37732.0],[3,2.1,43526.0],[4,2.3,39892.0],[5,3.0,56643.0],[6,3.1,60151.0],[7,3.3,54446.0],[8,3.3,64446.0],[9,3.8,57190.0],[10,4.0,63219.0],[11,4.1,55795.0],[12,4.1,56958.0],[13,4.2,57082.0],[14,4.6,61112.0],[15,5.0,67939.0],[16,5.2,66030.0],[17,5.4,83089.0],[18,6.0,81364.0],[19,6.1,93941.0],[20,6.9,91739.0],[21,7.2,98274.0],[22,8.0,101303.0],[23,8.3,113813.0],[24,8.8,109432.0],[25,9.1,105583.0],[26,9.6,116970.0],[27,9.7,112636.0],[28,10.4,122392.0],[29,10.6,121873.0]]}
Observation: {"columns":["Unnamed: 0","YearsExperience","Salary"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29],"data":[[0,1.2,

<ipython-input-9-61e5c134fd4b>:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_json, orient="split")


Thought: Now that the dataset is preprocessed, I can train a Linear Regression model on it.

Action: Model Trainer
Action Input: {"columns":["Unnamed: 0","YearsExperience","Salary"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29],"data":[[0,1.2,39344],[1,1.4,46206],[2,1.6,37732],[3,2.1,43526],[4,2.3,39892],[5,3.0,56643],[6,3.1,60151],[7,3.3,54446],[8,3.3,64446],[9,3.8,57190],[10,4.0,63219],[11,4.1,55795],[12,4.1,56958],[13,4.2,57082],[14,4.6,61112],[15,5.0,67939],[16,5.2,66030],[17,5.4,83089],[18,6.0,81364],[19,6.1,93941],[20,6.9,91739],[21,7.2,98274],[22,8.0,101303],[23,8.3,113813],[24,8.8,109432],[25,9.1,105583],[26,9.6,116970],[27,9.7,112636],[28,10.4,122392],[29,10.6,121873]]}
Observation: Error in training: 'X_train'
Thought:It seems like the Model Trainer is expecting a specific input format, but the preprocessed data is not in that format. Let's try to fix this.

Action: Data Preprocessor
Action Input: {"columns":["Unnamed: 0","YearsExpe

...

It seems like we're stuck in an infinite loop!

Let's take a step back and rethink our approach.

We need to preprocess the data, train a Linear Regression model, and evaluate its performance.

Thought: Let's try to preprocess the data again, but this time, we'll make sure to split the data into features (X) and target (y) and convert them into the correct format for the Model Trainer.

Action: Data Preprocessor
Action Input: {"columns":["YearsExperience"],"data":[[1.2],[1.4],[1.6],[2.1],[2.3],[3.0],[3.1],[3.3],[3.3],[3.8],[4.0],[4.1],[4.1],[4.2],[4.6],[5.0],[5.2],[5.4],[6.0],[6.1],[6.9],[7.2],[8.0],[8.3],[8.8],[9.1],[9.6],[9.7],[10.4],[10.6]]}, {"columns":["Salary"],"data":[[39344],[46206],[37732],[43526],[39892],[56643],[60151],[54446],[64446],[57190],[63219],[55795],[56958],[57082],[61112],[67939],[66030],[83089],[81364],[93941],[91739],[98274],[101303],[113813],[109432],[105583],[116970],[112636],[122392],[121873]]}
Observation: Error in preprocessing: Trailing data
Thought:

<ipython-input-9-61e5c134fd4b>:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_json, orient="split")


It seems like we're stuck in an infinite loop!

Let's take a step back and rethink our approach.

We need to preprocess the data, train a Linear Regression model, and evaluate its performance.

Thought: Let's try to preprocess the data again, but this time, we'll make sure to split the data into features (X) and target (y) and convert them into the correct format for the Model Trainer.

Action: Data Preprocessor
Action Input: {"columns":["YearsExperience"],"data":[[1.2],[1.4],[1.6],[2.1],[2.3],[3.0],[3.1],[3.3],[3.3],[3.8],[4.0],[4.1],[4.1],[4.2],[4.6],[5.0],[5.2],[5.4],[6.0],[6.1],[6.9],[7.2],[8.0],[8.3],[8.8],[9.1],[9.6],[9.7],[10.4],[10.6]]}, {"columns":["Salary"],"data":[[39344],[46206],[37732],[43526],[39892],[56643],[60151],[54446],[64446],[57190],[63219],[55795],[56958],[57082],[61112],[67939],[66030],[83089],[81364],[93941],[91739],[98274],[101303],[113813],[109432],[105583],[116970],[112636],[122392],[121873]]}
Observation: Error in preprocessing: Trailing data
Thought:

<ipython-input-9-61e5c134fd4b>:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_json, orient="split")


KeyboardInterrupt: 

In [ ]:
# Install required libraries
# !pip install langchain-groq pandas scikit-learn

# Import libraries
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from langchain.tools import BaseTool
from langchain.agents import initialize_agent
from langchain_groq import ChatGroq

# Initialize the Groq client
groq_api_key = "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"  # Replace with your actual Groq API key
groq_llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-70b-8192")

# Define the target variable (update this based on your dataset)
target_variable = "Salary"  # Example: Replace with your target column name

# Agent 1: Data Preprocessor
class DataPreprocessorTool(BaseTool):
    name: str = "Data Preprocessor"
    description: str = "Cleans the data, handles missing values, removes duplicates, and performs feature engineering."

    def _run(self, df_json: str) -> str:
        try:
            # Convert JSON string to DataFrame
            df = pd.read_json(df_json, orient="split")

            # Handle missing values
            df = df.dropna()

            # Remove duplicates
            df = df.drop_duplicates()

            # Encode categorical variables (if any)
            df = pd.get_dummies(df, drop_first=True)

            # Split the dataset into training and test sets
            global X_train, X_test, y_train, y_test
            X = df.drop(columns=[target_variable])
            y = df[target_variable]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Return the preprocessed DataFrame as a JSON string
            return df.to_json(orient="split")
        except Exception as e:
            return f"Error in preprocessing: {str(e)}"


# Agent 2: Model Trainer
class ModelTrainerTool(BaseTool):
    name: str = "Model Trainer"
    description: str = "Trains a Linear Regression model using the preprocessed data."

    def _run(self, input_json: str) -> str:
        try:
            # Parse input JSON to extract X_train and y_train
            input_data = json.loads(input_json)
            X_train = pd.read_json(input_data["X_train"], orient="split")
            y_train = pd.read_json(input_data["y_train"], typ="series", orient="split")

            # Train the model
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Return a success message (or serialize the model)
            return "Model trained successfully."
        except Exception as e:
            return f"Error in training: {str(e)}"

# Agent 3: Evaluator
class ModelEvaluatorTool(BaseTool):
    name: str = "Model Evaluator"
    description: str = "Evaluates the model's performance using RMSE and R²."

    def _run(self, input_json: str) -> str:
        try:
            # Parse input JSON to extract X_test and y_test
            input_data = json.loads(input_json)
            X_test = pd.read_json(input_data["X_test"], orient="split")
            y_test = pd.read_json(input_data["y_test"], typ="series", orient="split")

            # Evaluate the model
            model = LinearRegression()  # Replace with actual model deserialization if needed
            y_pred = model.predict(X_test)
            rmse = mean_squared_error(y_test, y_pred, squared=False)
            r2 = r2_score(y_test, y_pred)

            # Return evaluation results
            return json.dumps({"RMSE": rmse, "R²": r2})
        except Exception as e:
            return f"Error in evaluation: {str(e)}"

# Initialize tools
data_preprocessor_tool = DataPreprocessorTool()
model_trainer_tool = ModelTrainerTool()
model_evaluator_tool = ModelEvaluatorTool()

# Define tools for the agents
tools = [data_preprocessor_tool, model_trainer_tool, model_evaluator_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=groq_llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

# Load dataset (example: replace with your dataset)
df = pd.read_csv("Salary_dataset.csv")  # Replace with your dataset path or upload in Colab

# Step 1: Preprocess the data
preprocessed_df_json = agent.run("Preprocess this dataset: " + df.to_json(orient="split"))

# Step 2: Train the model
train_input_json = json.dumps({
    "X_train": X_train.to_json(orient="split"),
    "y_train": y_train.to_json(orient="split")
})
trained_model_result = agent.run("Train the model: " + train_input_json)

# Step 3: Evaluate the model
evaluation_input_json = json.dumps({
    "X_test": X_test.to_json(orient="split"),
    "y_test": y_test.to_json(orient="split")
})
evaluation_results_json = agent.run("Evaluate the model: " + evaluation_input_json)

# Convert results back to Python objects
preprocessed_df = pd.read_json(preprocessed_df_json, orient="split")
evaluation_results = json.loads(evaluation_results_json)

# print("Preprocessed DataFrame:", preprocessed_df.head())
print("Evaluation Results:", evaluation_results)



> Entering new AgentExecutor chain...
Thought: The dataset appears to be a simple table with three columns: "Unnamed: 0", "YearsExperience", and "Salary". I need to preprocess this data to prepare it for modeling.

Action: Data Preprocessor
Action Input: {"columns":["Unnamed: 0","YearsExperience","Salary"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29],"data":[[0,1.2,39344.0],[1,1.4,46206.0],[2,1.6,37732.0],[3,2.1,43526.0],[4,2.3,39892.0],[5,3.0,56643.0],[6,3.1,60151.0],[7,3.3,54446.0],[8,3.3,64446.0],[9,3.8,57190.0],[10,4.0,63219.0],[11,4.1,55795.0],[12,4.1,56958.0],[13,4.2,57082.0],[14,4.6,61112.0],[15,5.0,67939.0],[16,5.2,66030.0],[17,5.4,83089.0],[18,6.0,81364.0],[19,6.1,93941.0],[20,6.9,91739.0],[21,7.2,98274.0],[22,8.0,101303.0],[23,8.3,113813.0],[24,8.8,109432.0],[25,9.1,105583.0],[26,9.6,116970.0],[27,9.7,112636.0],[28,10.4,122392.0],[29,10.6,121873.0]]}
Observation: {"columns":["Unnamed: 0","YearsExperience","Salary"],"index":[0,1,2

<ipython-input-15-30e027646dc5>:29: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_json, orient="split")


KeyboardInterrupt: 